In [1]:
from functools import partial
from pathlib import Path
import pickle

import numpy as np
import jax
import jax.numpy as jnp
from flax.training.train_state import TrainState
from flax import serialization
import optax
import pandas as pd

from common.cell import to_rgba, make_circle_masks
from common.pool import Pool
from common.nca import NCA
from common.vae import vae_dict, vae_loss
from common.utils import Config, load_face, visualize_nca, plot_loss, export_model

import tqdm
import hydra
from hydra.core.config_store import ConfigStore
from omegaconf import OmegaConf
import wandb

In [31]:
import matplotlib.pyplot as plt
height, width = 64, 64
h = jnp.arange(1, height+1)
w = jnp.arange(1, width+1)